# Utilização do Snorkel 
--- 
### 01. Import da base

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
books_df = pd.read_csv('../data/livros.csv')
books_df.columns = books_df.columns.map(str.lower)
books_df.head(5)

,titulo,genero
0,Iniciação ao Estudo da Administração,Administracao
1,Iniciação a Administração geral,Administracao
2,Iniciação a Administração de pessoal,Administracao
3,Administração de Materiais,Administracao
4,Gestão Ambiental na Empresa,Administracao


--- 

### 02. Análise da Base 

"Uma maneira de compreender o conteúdo de um texto é realizar uma análise das palavras mais essenciais, ou seja, aquelas que ocorrem com maior frequência e possuem maior significado para o tema abordado. 

Devido à natureza da base de dados, os títulos geralmente consistem em poucas palavras e têm uma quantidade limitada de exemplos, o que possibilita uma análise mais subjetiva para identificar quais palavras podem ser relevantes na classificação do tópico principal."

In [3]:
import re
from unidecode import unidecode 
import nltk
from nltk.corpus import stopwords
import spacy

# Download Portuguese stopwords from the NLTK library
nltk.download('stopwords')

def remove_special_characters_and_stopwords(text):
    """
    Removes special characters (except Latin usual characters),
    tokenizes the text, and removes Portuguese stopwords.
    
    Args:
        text (str): The input text to process.
        
    Returns:
        list of str: A list of cleaned and tokenized words without stopwords.
    """
    # Remove special characters (except Latin usual characters)
    clean_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove accents and diacritics from the text (e.g., converting é to e)
    clean_text = unidecode(clean_text)
    
    # Tokenize the cleaned text into a list of words
    clean_text = clean_text.strip().split()

    # Get the list of Portuguese stopwords
    stopwords_list = set(stopwords.words('portuguese'))
    
    # Remove stopwords from the tokenized words
    clean_text = [word.lower() for word in clean_text if word.lower() not in stopwords_list]

    return clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
books_df['alt_title'] = books_df['titulo'].map(remove_special_characters_and_stopwords)

In [5]:
list(books_df['genero'].unique())

['Administracao',
 'Artes',
 'Biologia',
 'Geografia',
 'Historia',
 'Literatura',
 'Matematica']

In [6]:
# Select the 'genero' and 'alt_title' columns from the DataFrame
# and explode the 'alt_title' column to create multiple rows for each title
# Then we can see the most relevant words for each genero
(
 books_df[['genero', 'alt_title']]
 .explode('alt_title')
 
 # Group the data by 'genero', reset the index, and calculate the value counts
 .groupby(by='genero', as_index=False)
 .value_counts()
 
 # Sort the values in descending order first by 'genero' and then by 'count'
 .sort_values(by=['genero', 'count'], ascending=False)
 
 # Group the sorted data by 'genero' and select the top 15 elements for each group
 .groupby('genero')
 .head(100)

# Change the word inside the double quotes for selection
).query('genero=="Historia"')


,genero,alt_title,count
800,Historia,historia,154
801,Historia,brasil,79
802,Historia,geral,30
803,Historia,arte,22
804,Historia,guerra,19
...,...,...,...
895,Historia,documentos,3
896,Historia,indios,3
897,Historia,indigenas,3
898,Historia,leste,3


#### 02.01. Conclusões de análises: 

Iremos utilizar o pacote Snorkel para data augmentation, que utiliza conceitos simples 
para fazer uma inferência heurística. Após a remoção das stop words e limpeza das palavras, 
é possível selecionar algumas palavras-chave que podem fortemente indicar um gênero.

Essas análises e seleções não possuem critérios exatos. Portanto, caso você esteja vendo 
o código, é possível que você ache uma ou outra palavra mais relevante do que as selecionadas.



| Administração | Artes   | Biologia   | Geografia  | História    | Literatura | Matemática  |
|---------------|---------|------------|------------|-------------|------------|-------------|
| administracao | museu   | biologia   | geografia  | historia    | literatura | matematica  |
| organizacoes  | pintura | seres      | geografico | guerra      | texto      | fundamentos |
| organizacao   | arte    | genetica   | sociedade  | revolucao   | portuguesa | geometria   |
| gestao        | teatro  | vida       | regiao     | anos        | leitura    | calculo     |
| empresa       | museum  | biologicas | territorio | civilizacao | gramatica  | analitica   |
|               | gallery | evolucao   |            | antiga      |            | financeira  |

--- 
## 03. Snorkel

O pacote Snorkel do Python é um sistema que permite a geração rápida de dados de treinamento com supervisão fraca. Ele foi criado para automatizar o processo de criação e gerenciamento de dados de treinamento, permitindo que os usuários rotulem, construam e gerenciem dados de treinamento programaticamente.

In [7]:
LABELS = {'Historia' : 0, 'Administração' : 1, 'Geografia' : 2, 
          'Biologia' : 3, 'Matemática' : 4, 'Artes': 5, 'Literatura': 6}

#### 03.01 Teste e treino para o modelo de Weak Supervision

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(books_df, test_size=0.25, random_state=42, stratify=books_df['genero'])

In [9]:
x_train['alt_title'] = x_train['alt_title'].map(lambda x: ' '.join(x))
x_test['alt_title'] = x_test['alt_title'].map(lambda x: ' '.join(x))

#### 03.02. Função de Rotulação 

A função labeling_function no Snorkel é usada para criar funções de rotulagem fracas que 
atribuem rótulos a instâncias de dados. Essas funções de rotulagem são frequentemente 
escritas por especialistas humanos e são baseadas em regras. Essas funções são usadas para 
gerar rótulos aproximados ou fracamente supervisionados para os dados.



In [10]:
from snorkel.labeling import labeling_function

# Define a labeling function for the "Historia" label
@labeling_function()
def lf_historia(words: str):
    """
    This function takes in a list of words and returns the "Historia" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Historia" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Historia" label
    historia_keywords = ["guerra", "revol", "anos", "civiliz", "antiga", 
                         "hist", "arqueo", "arquitetura", "cultura", "brasil", "polit",
                         "socie", "socioe", "amer", "geral", "indep", "imperi", "repub", 
                         "reforma", "mediev", "moderna", "contempo", "europeia","idade", 
                         "colonial", "nacion", "africa", "colon", "descobri", "escrav", 
                         "crise", 'ociden', "doc", "antig"]    
    # Check if any of the keywords are present in the input words
    for word in historia_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Historia']
    # Return -1 to represent "no label" if none of the keywords match
    return -1 

@labeling_function()
def lf_administracao(words: str):
    """
    This function takes in a list of words and returns the "Administração" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Administração" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Administração" label
    
    administracao_keywords = ["adm", "org", "gest", "empr", "neg", "econ", "controle"]
    # Check if any of the keywords are present in the input words
    for word in administracao_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Administracao']
    # Return -1 to represent "no label" if none of the keywords match    return -1
    return -1

# Define a labeling function for the "Geografia" label
@labeling_function()
def lf_geografia(words: str):
    """
    This function takes in a list of words and returns the "Geografia" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Geografia" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Geografia" label
    geografia_keywords = ["geo", "socio", "antropo", "regiao", "terr", "rural", "urban", 
                          'mund', 'atlas', 'mapa', 'carto', 'clima', 'ambien', 'ecolo', 'hidro',]
    # Check if any of the keywords are present in the input words
    for word in geografia_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Geografia']
    # Return -1 to represent "no label" if none of the keywords match
    return -1


# Define a labeling function for the "Biologia" label
@labeling_function()
def lf_biologia(words: str):
    """
    This function takes in a list of words and returns the "Biologia" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Biologia" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Biologia" label
    biologia_keywords = ["bio", "seres", "vida", "evol", "genet", "medic", "saude", 
                         "nutri", "fisio", "enferm", "farma", "veteri","odonto", "psico"]
    # Check if any of the keywords are present in the input words
    for word in biologia_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Biologia']
    # Return -1 to represent "no label" if none of the keywords match
    return -1

# Define a labeling function for the "Literatura" label
@labeling_function()
def lf_literatura(words: str):
    """
    This function takes in a list of words and returns the "Literatura" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Literatura" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Literatura" label
    literatura_keywords = ["litera", "texto", "portuguesa", "leit", "grama", "poesia", "poema"]
    # Check if any of the keywords are present in the input words
    for word in literatura_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Literatura']
    # Return -1 to represent "no label" if none of the keywords match
    return -1

# Define a labeling function for the "Artes" label
@labeling_function()
def lf_artes(words: str):
    """
    This function takes in a list of words and returns the "Artes" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Artes" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Artes" label
    artes_keywords = ["museu", "cinema", "filme", "museum", "gallery",
                      "art", "pintur", "escult", "music", "teatr", "danc"]
    # Check if any of the keywords are present in the input words
    for word in artes_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Artes']
    # Return -1 to represent "no label" if none of the keywords match
    return -1

# Define a labeling function for the "Matemática" label
@labeling_function()
def lf_matematica(words: str):
    """
    This function takes in a list of words and returns the "Matemática" label if any of the keywords for this label are present
    in the input words. If none of the keywords match, it returns -1 to represent "no label".
    
    Parameters:
    words (list): A list of strings representing the words to be checked for keywords.
    
    Returns:
    int: An integer representing the "Matemática" label, or -1 if no match is found.
    """
    # Define a list of keywords for the "Matemática" label
    matematica_keywords = ["matema", "fundam", "calcu", "algebra", "geome", "estatis", 
                           "proba", "trigono", "logica", "fisica", 'analit', 'aplica', 
                           "medio", "grau", 'discre', 'numeri', 'vetor', 'equa', 'difer', 'integral']
    # Check if any of the keywords are present in the input words
    for word in matematica_keywords:
        if remove_special_characters_and_stopwords(word)[0] in ' '.join(words):
            # Return the corresponding label if a match is found
            return LABELS['Matematica']
    # Return -1 to represent "no label" if none of the keywords match
    return -1


In [11]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

# Define the missing variables
LABELS = {'Historia': 0, 'Administracao': 1, 'Geografia': 2, 'Biologia': 3, 
          'Literatura': 4, 'Artes': 5, 'Matematica': 6}

applier = PandasLFApplier(lfs=[lf_historia, lf_administracao, lf_geografia, lf_biologia, 
                               lf_literatura, lf_artes, lf_matematica])
L_train = applier.apply(df=x_train)


  0%|          | 4/872 [00:00<00:22, 39.10it/s]

100%|██████████| 872/872 [00:17<00:00, 50.53it/s]


In [12]:
label_model = LabelModel(cardinality=7, verbose=True)
label_model.fit(L_train=L_train, 
                n_epochs=10, 
                lr=0.01,
                log_freq=20, 
                seed=42)

x_train['label_inf'] = label_model.predict(L=L_train, tie_break_policy="abstain")


INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 10/10 [00:00<00:00, 512.53epoch/s]
INFO:root:Finished Training


In [13]:
import os
label_model_dir = '../model/label_model_books/'
if not os.path.exists(label_model_dir):
    os.makedirs(label_model_dir)


In [14]:
# Export do Modelo
label_model.save('../model/label_model_books.pkl')

#### 03.03. Função de Transformação

In [21]:
import nlpaug.augmenter.word as naw
from snorkel.augmentation import transformation_function

# Initialize augmenters
# synonym_aug = naw.SynonymAug(lang='por', aug_min=1)
context_aug = naw.ContextualWordEmbsAug(model_path='../model/bert/', aug_min=1)

def synonym_replacer(text, model=context_aug):
    """
    This function takes in a string of text and performs data augmentation using the specified augmentation technique.
    
    Parameters:
    text (str): A string of text to be augmented.
    
    Returns:
    str: The augmented text.
    """
    # Define the augmentation technique
    aug = model
    
    return aug.augment(text)[0]

@transformation_function()
def tf_synonym(df_row): 
    # Assuming that 'alt_title' is the column containing text
    df_row['alt_title'] = synonym_replacer(df_row.alt_title)
    return df_row


In [22]:
from snorkel.augmentation import ApplyOnePolicy, PandasTFApplier

tf_policy = ApplyOnePolicy(n_per_original=3, keep_original=True)
tf_applier = PandasTFApplier([tf_synonym], tf_policy)

x_train_aug = tf_applier.apply(x_train)

100%|██████████| 872/872 [10:01<00:00,  1.45it/s]


In [23]:
L_train = applier.apply(df=x_train_aug.drop('label_inf', axis=1))

100%|██████████| 3488/3488 [01:06<00:00, 52.73it/s]


In [24]:

label_model = LabelModel(cardinality=7, verbose=True)
label_model.fit(L_train=L_train, 
                n_epochs=100, 
                lr=0.1,
                log_freq=20, 
                seed=42)


INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/100 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.124]
INFO:root:[20 epochs]: TRAIN:[loss=0.013]
INFO:root:[40 epochs]: TRAIN:[loss=0.001]
INFO:root:[60 epochs]: TRAIN:[loss=0.001]
100%|██████████| 100/100 [00:00<00:00, 650.48epoch/s]
INFO:root:Finished Training


In [25]:
x_train_aug['label_inf'] = label_model.predict(L=L_train, tie_break_policy="abstain")
x_train_aug['label'] = x_train_aug['genero'].map(LABELS)

In [26]:
pd.crosstab(x_train_aug['label'], x_train_aug['label_inf'], margins=True)

label_inf,-1,0,1,2,3,4,5,6,All
label,,,,,,,,,
0,548,706,24,41,29,5,25,14,1392
1,166,71,191,3,11,6,10,14,472
2,87,155,7,136,4,1,1,5,396
3,153,56,10,25,150,8,1,5,408
4,51,64,0,0,0,40,0,5,160
5,131,50,0,3,0,1,74,1,260
6,132,16,5,43,8,4,8,184,400
All,1268,1118,237,251,202,65,119,228,3488


In [27]:
from snorkel.slicing import slicing_function, PandasSFApplier 

@slicing_function()
def sf_historia(x):
    return 1 if "histori" in x.alt_title.lower() else 0

@slicing_function()
def sf_administracao(x):
    return 1 if "administra" in x.alt_title.lower() else 0

@slicing_function()
def sf_geografia(x):
    return 1 if "geogra" in x.alt_title.lower() else 0

@slicing_function()
def sf_biologia(x):
    return 1 if "biolo" in x.alt_title.lower() else 0

@slicing_function()
def sf_literatura(x):
    return 1 if "litera" in x.alt_title.lower() else 0

@slicing_function()
def sf_artes(x):
    return 1 if "arte" in x.alt_title.lower() else 0

@slicing_function()
def sf_matematica(x):
    return 1 if "matem" in x.alt_title.lower() else 0


sf_applier = PandasSFApplier([sf_historia, sf_administracao, sf_geografia, 
                              sf_biologia, sf_literatura, sf_artes, sf_matematica])
slice_labels = sf_applier.apply(x_train_aug)

# slice_labels = pd.DataFrame(slice_labels)

100%|██████████| 3488/3488 [00:00<00:00, 25134.02it/s]


In [28]:
x_train_final = pd.concat([x_train_aug.reset_index(drop=True), 
                           pd.DataFrame(slice_labels)]
                           , axis=1)

#### Export das bases

In [29]:
x_train['label'] = x_train['genero'].map(LABELS)
x_train_aug['label'] = x_train_aug['genero'].map(LABELS)
x_test['label'] = x_test['genero'].map(LABELS)

In [30]:
x_train.drop(['titulo', 'genero'], axis=1, inplace=True)
x_train_aug.drop(['titulo', 'genero'], axis=1, inplace=True)
x_train_final.drop(['titulo', 'genero'], axis=1, inplace=True)
x_test.drop(['titulo', 'genero'], axis=1, inplace=True)

In [31]:
x_train.to_csv('../data/processed/train_og.csv', index=False)
x_train_aug.to_csv('../data/processed/train_aug.csv', index=False)
x_train_final.to_csv('../data/processed/train_final.csv', index=False)
x_test.to_csv('../data/processed/test.csv', index=False)